In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1623728/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_1623728/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_16_5,0.584485,0.046998,-0.869677,0.439203,0.410453,1.910473,1.378724,0.417726,2.126462e+14,0.640666,0.434192,0.650756,35.780988,72.745571,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1,model_1_9_6,0.574328,0.035375,-0.940811,0.401724,0.420486,1.933773,1.431179,0.100502,2.304874e+14,0.648449,0.420362,0.658661,35.732690,72.697273,"Hidden Size=[4], regularizer=0.02, learning_ra..."
2,model_1_16_0,0.586253,0.035109,-0.912201,0.470703,0.408706,1.934306,1.410082,0.394262,2.152410e+14,0.639301,0.436600,0.649370,35.789517,72.754100,"Hidden Size=[4], regularizer=0.02, learning_ra..."
3,model_1_12_3,0.589510,0.033070,-0.898497,0.468022,0.405489,1.938395,1.399976,0.892088,2.136907e+14,0.636780,0.441035,0.646809,35.805325,72.769909,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4,model_1_16_9,0.588964,0.030546,-0.908803,0.421706,0.406028,1.943455,1.407576,0.430759,2.092278e+14,0.637204,0.440291,0.647239,35.802665,72.767248,"Hidden Size=[4], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,model_2_29_0,0.607417,-0.715310,-3.547118,0.157225,0.387800,3.438665,3.353104,0.266994,6.085625e+13,0.622736,0.189506,0.632543,67.894531,139.649310,"Hidden Size=[8], regularizer=0.02, learning_ra..."
486,model_2_28_9,0.525681,-0.738898,-3.727661,-2.443485,0.468540,3.485951,3.486239,1.595163,4.833173e+13,0.684500,0.020762,0.695280,67.516268,139.271048,"Hidden Size=[8], regularizer=0.02, learning_ra..."
487,model_3_15_6,0.757092,-0.894362,-2.648797,-0.257282,0.239949,3.797608,2.690670,0.640041,5.347272e+13,0.489846,-0.036409,0.497560,100.854659,207.399635,"Hidden Size=[12], regularizer=0.02, learning_r..."
488,model_3_15_9,0.771733,-6.764902,-38.236998,-0.333430,0.225486,15.566219,28.933871,0.678805,4.479837e+13,0.474853,0.026062,0.482332,100.978997,207.523973,"Hidden Size=[12], regularizer=0.02, learning_r..."
